# `Studio reversibilità catene P e catene Q`

## `Librerie`

In [ ]:
import generazione_catene, metodi_stazionaria, 

## `Test Reversibilità`

### Confronto tra mio metodo e `pydctm`
Confrontiamo la correttezza di reversibilità tra il nostro metodo e quello interno alla libreria `pydctm`

In [ ]:
test_confronto_reversibilita(n_matrici=100, n=100)

#### Ora confrontiamo i tempi per il controllo della reversibilità e confrontiamo il nostro metodo e quello di `pydctm`

In [ ]:
# Testa nostro metodo su P reversibili
ris_nostro = calcola_tempi("nostro", C=1000, n_punti=50, n_matrici=1)

# Testa PyDTMC su P reversibili
ris_pydtmc = calcola_tempi("pydtmc", C=100, n_punti=50, n_matrici=1)

In [ ]:
# Grafico
plt.figure(figsize=(18,8))
plt.plot([r[0] for r in ris_nostro], [r[1] for r in ris_nostro], 'o-', label="Nostro check")
plt.plot([r[0] for r in ris_pydtmc], [r[1] for r in ris_pydtmc], 's-', label="PyDTMC check")
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Dimensione matrice $n$', fontsize=12)
plt.ylabel('Tempo medio (secondi)', fontsize=12)
plt.title('Confronto velocità verifica reversibilità su matrici P reversibili non banali', fontsize=14)
plt.grid(True, which="both", ls="--")
plt.legend()
plt.savefig("P_nostro_vs_pydctm.svg", dpi=300, format="svg", bbox_inches = "tight")
plt.show()

### Confronto tra matrici P e matrici Q ben condizionate per la uniformizzazione

In [ ]:
# Testa su P reversibili
ris_nostro = calcola_tempi("nostro", C=100, n_punti=50, n_matrici=1)

# Testa su Q reversibili "facili"
ris_pydtmc = calcola_tempi("nostro", C=100, n_punti=50, n_matrici=1, discrete=False, forzata=True)

In [ ]:
# Grafico
plt.figure(figsize=(18,8))
plt.plot([r[0] for r in ris_nostro], [r[1] for r in ris_nostro], 'o-', label="Nostro check su P")
plt.plot([r[0] for r in ris_pydtmc], [r[1] for r in ris_pydtmc], 's-', label="Nostro check su Q")
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Dimensione matrice $n$', fontsize=12)
plt.ylabel('Tempo medio (secondi)', fontsize=12)
plt.title('Confronto velocità verifica reversibilità su matrici P e Q (ben condizionate) reversibili non banali', fontsize=14)
plt.grid(True, which="both", ls="--")
plt.legend()
plt.savefig("P_Qbene.svg", dpi=300, format="svg", bbox_inches = "tight")
plt.show()

### Confronto tra Q ben condizionate e Q generate senza accortezza

In [ ]:
# Testa su Q reversibili
ris_nostro = calcola_tempi("nostro", C=70, n_punti=50, discrete=False)

# Testa su Q reversibili "facili"
ris_pydtmc = calcola_tempi("nostro", C=70, n_punti=50, discrete=False, forzata=True)

In [ ]:
# Grafico
plt.figure(figsize=(18,8))
plt.plot([r[0] for r in ris_nostro], [r[1] for r in ris_nostro], 'o-', label="Nostro check su Q generiche")
plt.plot([r[0] for r in ris_pydtmc], [r[1] for r in ris_pydtmc], 's-', label="Nostro check su Q ben condizionate")
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Dimensione matrice $n$', fontsize=12)
plt.ylabel('Tempo medio (secondi)', fontsize=12)
plt.title('Confronto velocità verifica reversibilità su matrici Q ben condizionate e non', fontsize=14)
plt.grid(True, which="both", ls="--")
plt.legend()
plt.savefig("Qbene_Qnonbene.svg", dpi=300, format="svg", bbox_inches = "tight")
plt.show()